In [42]:
import gym
import tensorflow as tf
import numpy as np 
import random

In [43]:
env = gym.make('Breakout-v0')

[2017-05-30 10:09:23,986] Making new env: Breakout-v0


In [44]:
batch_size = 4
num_actions = env.action_space.n
num_episodes = 1
ep = 0.1
init_replay_size = 100
num_iterations = 1
batch_size = 4

In [113]:
class Network() :
    
    def __init__(self, num_actions, scope='Network', summary_dir=None) :
        self.num_actions = num_actions 
        
        self.fw = None
        if summary_dir :
            self.fw = tf.summary.FileWriter(summary_dir)
        
        self.build_network()
        
    def build_network(self) :
        self.img = tf.placeholder(tf.float32, shape=[None, 84, 84, 1], name="X")
        self.action_ix = tf.placeholder(tf.int32, shape=[None], name='action_ix')
        self.td_target = tf.placeholder(tf.float32, shape=[None], name='Y')

        conv1 = tf.layers.conv2d(inputs=self.img, filters=32, kernel_size=[8, 8], 
                                 strides=(4, 4), padding='SAME', activation=tf.nn.relu)
        conv2 = tf.layers.conv2d(inputs=conv1, filters=64, kernel_size=[4, 4], 
                                 strides=(2, 2), padding='SAME', activation=tf.nn.relu)
        conv3 = tf.layers.conv2d(inputs=conv2, filters=64, kernel_size=[3, 3], 
                                 strides=(1, 1), padding='SAME', activation=tf.nn.relu)

        conv3_flat = tf.contrib.layers.flatten(conv3)
        dense = tf.layers.dense(conv3_flat, units=1024)
        self.action_vals = tf.layers.dense(dense, units=self.num_actions)

        batch_size = tf.shape(self.img)[0]
        gather_ix = tf.range(batch_size) * self.num_actions + self.action_ix
        pred_action_val = tf.gather(tf.reshape(self.action_vals, [-1]), gather_ix)
        
        self.loss = tf.reduce_mean(tf.squared_difference(self.td_target, pred_action_val))
        optimizer = tf.train.RMSPropOptimizer(0.00025, 0.99, 0.0, 1e-6)
        self.global_step = tf.contrib.framework.get_global_step()
        self.train_op = optimizer.minimize(self.loss, global_step=self.global_step)
        
        tf.summary.scalar('loss', self.loss)
        self.summary_op = tf.summary.merge_all()
    
    def predict(self, sess, obs) :
        net_input = obs 
        if obs.ndim == 3 :
            net_input = np.expand_dims(obs, axis=0)
            
        action_values = sess.run(self.action_vals, {self.img:net_input})
        return action_values

    def update(self, sess, obs_t, A_ix, targets) :
        print(obs_t.dtype)
        summaries = sess.run([self.summary_op], \
                                      {self.img: obs_t, self.action_ix: A_ix, self.td_target: targets})
#         print(obs_t.dtype)
#         loss, _ = sess.run([self.loss, self.train_op], \
#                                       {self.img: obs_t, self.action_ix: A_ix, self.td_target: targets})
#         print('here')
        
#         if self.fw :
#             self.fw.add_summary(summaries, global_step)

In [91]:
raw_imgs = tf.placeholder(tf.int8, shape=[210, 160, 3], name="raw_imgs")
grayscale = tf.image.rgb_to_grayscale(raw_imgs)
resized = tf.image.resize_images(
                grayscale, [84, 84], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
resized = tf.cast(resized, tf.float32)

def preprocess(sess, obs) :
    return sess.run(resized, {raw_imgs:obs})

In [92]:
def init_greedy_pi(Qnet) :
    def pi(sess, obs) :

        Qs = Qnet.predict(sess, obs)
        greedy_action = np.argmax(Qs)

        dist = np.ones(num_actions) * ep/num_actions
        dist[greedy_action] += 1 - ep

        return np.random.choice(num_actions, p=dist)
    return pi

In [114]:
Q_net = Network(num_actions, scope='Q', summary_dir='./summary/')
# target_net = Network(num_actions, scope='Q_target')
    
pi = init_greedy_pi(Q_net)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [94]:
# populate initial replay memory

# list of obs, action, reward
replay_memory = []

while len(replay_memory) < init_replay_size :
    
    done = False
    obs_t = env.reset()
    obs_t = preprocess(sess, obs_t)
    while not done :
        A_t = pi(sess, obs_t)
        obs_tp1, R_tp1, done, info = env.step(A_t)
        replay_memory.append([obs_t, A_t, R_tp1])
        obs_t = preprocess(sess, obs_tp1)

In [115]:
for ix in range(num_episodes) :
    
    done = False
    obs_t = env.reset()
    obs_t = preprocess(sess, obs_t)
    while not done :
        A_t = pi(sess, obs_t)
        obs_tp1, R_tp1, done, info = env.step(A_t)
        
        replay_memory.append([obs_t, A_t, R_tp1])
        # sample minibatch from replay_memory
        replay_sample = random.sample(replay_memory, batch_size)
        obs_batch, action_batch, reward_batch = map(np.array, zip(*replay_sample))

        target_Qs = target_net.predict(sess, obs_batch)
        max_Qs = np.amax(target_Qs, axis=0)
        TD_targets = reward_batch + max_Qs

        Q_net.update(sess, obs_batch, action_batch, TD_targets)
        obs_t = preprocess(sess, obs_tp1)

float32


InvalidArgumentError: You must feed a value for placeholder tensor 'X_3' with dtype float
	 [[Node: X_3 = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'X_3', defined at:
  File "/Users/shaayaansayed/anaconda2/envs/rl/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/shaayaansayed/anaconda2/envs/rl/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/shaayaansayed/anaconda2/envs/rl/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/shaayaansayed/anaconda2/envs/rl/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/shaayaansayed/anaconda2/envs/rl/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/shaayaansayed/anaconda2/e